In [1]:
import numpy as np
import os
import pandas as pd
import xml.etree.ElementTree as etree
import copy 
import datetime 
data_path = '../../../code_data/ohio/ohio_raw_data/'
save_path = '../../../code_data/ohio/coldstart_fl_analysis'
pid_list_2018 = [559, 563, 570, 575, 588, 591]
pid_list_2020 = [540, 544, 552, 567, 584, 596]
pid_list = pid_list_2018 + pid_list_2020
attri_list = [
    'glucose_level',
    'finger_stick',
    'basal',
    'temp_basal',
    'bolus',
    'meal',
    'sleep',
    'work',
    'stressors',
    'hypo_event',
    'illness',
    'exercise',
    'basis_heart_rate',
    'basis_gsr',
    'basis_skin_temperature',
    'basis_air_temperature',
    'basis_steps',
    'basis_sleep',
    'acceleration',
]

In [2]:

# load data from xml files
def load_xml(is_train):
    pid_attri2data = {}
    ending = '-ws-training.xml' if is_train else '-ws-testing.xml'

    # loop for pids
    for pid in pid_list:
        xml_path = os.path.join(data_path, f'{pid}' + ending)
        tree = etree.parse(xml_path)
        root = tree.getroot()
        # loop for attributes
        for attri in attri_list:
            try:
                temp = [event.attrib for event in root.find(attri).findall('event')]
            except:
                continue
            if len(temp) == 0:
                continue
            # change data to dataframe
            temp_pd = pd.DataFrame(temp)
            for col in temp_pd.columns:
                try:
                    temp_pd[col] = pd.to_datetime(temp_pd[col], dayfirst=True)
                except:
                    try:
                        temp_pd[col] = temp_pd[col].astype(float)
                    except:
                        pass
            rename_cols = {}
            for col in temp_pd.columns:
                rename_cols[col] = f'{attri}_{col}'
            temp_pd.rename(columns=rename_cols, inplace=True)

            if attri == 'basis_sleep':
                temp_pd.rename(columns={f'{attri}_tbegin': f'{attri}_ts_begin', f'{attri}_tend': f'{attri}_ts_end'}, inplace=True)
            
            # this part is time consuming
            if attri in ['glucose_level', 'finger_stick', 'basal', 
            'meal', 'stressors', 'exercise', 'basis_gsr',
             'basis_skin_temperature', 'acceleration', 'basis_heart_rate', 'basis_air_temperature', 'basis_steps']:
                temp_pd.set_index(f'{attri}_ts', inplace=True)
            elif attri in ['hypo_event']:
                pid_attri2data[(pid, attri)] = temp_pd
                continue
            else:

                temp_pd.set_index(f'{attri}_ts_begin', inplace=True)


            pid_attri2data[(pid, attri)] = temp_pd
    return pid_attri2data

In [3]:
# load training data
train_pid_attri2data = load_xml(is_train=True)

# load testing data
test_pid_attri2data = load_xml(is_train=False)

In [4]:
pid2regular_df = {}
for pid in pid_list:
    if pid in pid_list_2020:
        pid2regular_df[pid] = pd.concat([train_pid_attri2data[(pid, 'glucose_level')], test_pid_attri2data[(pid, 'glucose_level')].iloc[12:]])
        
    else:
        pid2regular_df[pid] = pd.concat([train_pid_attri2data[(pid, 'glucose_level')], test_pid_attri2data[(pid, 'glucose_level')]])
    pid2regular_df[pid].index.rename('ts', inplace=True)
    pid2regular_df[pid].rename(columns={'glucose_level_value':'glucose_level'}, inplace=True)


In [5]:
# add time features

def add_time_attributes(pid2data):
    for pid in pid2data:
        data = pid2data[pid]

        temp = data.index.to_frame().iloc[:, 0].dt
        
        data['day_of_week'] = temp.dayofweek
        data['day_of_week'] = data['day_of_week'].astype(np.float64)

        data['hour'] = temp.hour
        data['hour'] = data['hour'].astype(np.float64)

        data['minute'] = temp.minute
        data['minute'] = data['minute'].astype(np.float64)

        
        data['timestamp'] = temp.hour * 3600 +\
                                temp.minute * 60 +\
                                temp.second
        data['timestamp'] = data['timestamp'].astype(np.float64)
        
        # new ————————————————————————
        seconds_in_day = 24*60*60

        data['sin_time'] = np.sin(2 * np.pi * data.timestamp / seconds_in_day)
        data['cos_time'] = np.cos(2 * np.pi * data.timestamp / seconds_in_day)
        data['sin_time'].astype(np.float64)
        data['cos_time'].astype(np.float64)
        # end ______________________
        
        data['datastamp'] = temp.year * 10000 + temp.month * 100 + temp.day
        
        
    return pid2data
pid2regular_df = add_time_attributes(pid2regular_df)
for pid in pid2regular_df:
    data = pid2regular_df[pid]
    data = data.loc[pd.notna(data['glucose_level'])]
    pid2regular_df[pid] = data

In [6]:
pid2regular_df.keys()

dict_keys([559, 563, 570, 575, 588, 591, 540, 544, 552, 567, 584, 596])

In [7]:
def f(x):
    return 1.509 * ((np.log(x))**1.084 - 5.381)

def cal_LBGI(x):
    x = f(x) if f(x) <= 0 else 0
    return 10 * x ** 2

def cal_HBGI(x):
    x = f(x) if f(x) > 0 else 0
    return 10 * x ** 2
count2count_list = {}
count2count_list['No. of Days'] = []
count2count_list['CV'] = []
count2count_list['TIR'] = []
count2count_list['TBR'] = []
count2count_list['TAR'] = []

count2count_list['LBGI'] = []
count2count_list['HBGI'] = []

count2count_list['Mean of CGM data'] = []
count2count_list['SD of CGM data'] = []
count2count_list['No. of CGM records'] = []

for pid in pid2regular_df:
    
    glucose_avg_by_day = pid2regular_df[pid][['glucose_level', 'datastamp']].groupby('datastamp').mean()
    
    count2count_list['No. of Days'].append(len(glucose_avg_by_day))
    
    glucose_mean = pid2regular_df[pid]['glucose_level'].mean()
    glucose_std = pid2regular_df[pid]['glucose_level'].std()
    count2count_list['Mean of CGM data'].append(glucose_mean)
    count2count_list['SD of CGM data'].append(glucose_std)
    
    count2count_list['No. of CGM records'].append(len(pid2regular_df[pid]['glucose_level']))
 
    count2count_list['CV'].append((glucose_std / glucose_mean) * 100)

    target_range_min = 70
    target_range_max = 180
    
    glucose_values = pid2regular_df[pid]['glucose_level']
    time_in_range = ((glucose_values >= target_range_min) & (glucose_values <= target_range_max)).sum()
    total_time_points = len(glucose_values)
    tir_percentage = (time_in_range / total_time_points) * 100
    count2count_list['TIR'].append(tir_percentage)
    
    
    time_below_range = ((glucose_values < target_range_min)).sum()
    tbr_percentage = (time_below_range / total_time_points) * 100
    count2count_list['TBR'].append(tbr_percentage)
    
    time_above_range = ((glucose_values > target_range_max)).sum()
    tar_percentage = (time_above_range / total_time_points) * 100
    count2count_list['TAR'].append(tar_percentage)
    
    LBGI = pid2regular_df[pid]['glucose_level'].apply(cal_LBGI)
    HBGI = pid2regular_df[pid]['glucose_level'].apply(cal_HBGI)

    count2count_list['LBGI'].append(LBGI.mean())
    count2count_list['HBGI'].append(HBGI.mean())
    


In [57]:
for k in count2count_list:

   print(k, np.mean(count2count_list[k]), np.std(count2count_list[k]))

No. of Days 54.333333333333336 2.8963578661637945
CV 36.634097365058715 3.700004276590946
TIR 63.544498049959834 9.696947988155983
TBR 3.3012934138380063 2.246487412298367
TAR 33.154208536202155 10.712025045214212
LBGI 0.8842483313817754 0.48253118052188665
HBGI 7.151049499689435 2.4536745083449674
Mean of CGM data 159.34808974005276 16.335885496183423
SD of CGM data 58.112530497108544 6.153701918755674
No. of CGM records 13871.75 1015.1701930382577


In [61]:
import xlwt

book = xlwt.Workbook(encoding='utf-8', style_compression=0)
sheet = book.add_sheet('Main', cell_overwrite_ok=True)

row_names = ['No. of Days', 'No. of CGM records', 'Mean of CGM data', 'SD of CGM data', 'TIR', 'TBR','TAR', 'CV', 'LBGI', 'HBGI'] 


for r_idx, r_name in enumerate(row_names):
    sheet.write(r_idx + 3, 0, r_name)
    
    if 'No' not in r_name:
        print_str = f'{np.mean(count2count_list[r_name]):.2f}({np.std(count2count_list[r_name]):.2f})'
    else:
        print_str = f'{int(np.mean(count2count_list[r_name]))}({int(np.std(count2count_list[r_name]))})'
    sheet.write(r_idx + 3, 1, print_str)
book.save(f'ohio.xls')


In [8]:
count2count_list['No. of CGM records']

[13310,
 14694,
 13727,
 14456,
 15431,
 13607,
 14831,
 13327,
 11432,
 13235,
 14803,
 13608]

In [63]:
age = [30, 50, 30, 30, 50, 70] + [50] * 6
np.mean(age), np.std(age)

(46.666666666666664, 11.055415967851333)